In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'/ECE_285_proj/
%cd DeepZip_Compression

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ECE_285_proj
/content/drive/My Drive/ECE_285_proj/DeepZip_Compression


In [2]:
!ls

arithmetic_coder.ipynb	input.txt	       main.ipynb   __pycache__
data_preparation.py	main-checkpoint.ipynb  my_model.h5  README.md


In [3]:
import sys
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers import LSTM, Flatten, Conv1D, LocallyConnected1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from math import sqrt
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from matplotlib import pyplot
import keras
from sklearn.preprocessing import OneHotEncoder
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
import numpy as np
import argparse
import os
from keras.callbacks import CSVLogger
import keras.backend as K
np.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.


## DATA PREP

In [4]:
import data_preparation
#Convert letters to integers
integer_encoded, params = data_preparation.process_data('input.txt')

15072434
{'A': 0, 'G': 1, 'T': 2, 'C': 3}
{0: 'A', 1: 'G', 2: 'T', 3: 'C'}
[[1]
 [3]
 [3]
 [2]
 [0]
 [0]
 [1]
 [3]
 [3]
 [2]]
GCCTAAGCCT


In [0]:
bs=128
time_steps=64
num_epochs=20

In [6]:
#Generate traning data and its lable 
X,Y_raw, Y = data_preparation.generate_single_output_data(integer_encoded[: bs * 90 + 95 ],bs, time_steps)


(11615, 1)
(11615,)
11520
(11520, 65)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Trainning

In [0]:
def custom_loss(y_true, y_pred):
        return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)        

In [0]:
alphabet_size = Y.shape[1]
DZ_model = Sequential()
DZ_model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(Flatten())
DZ_model.add(Dense(64, activation='relu'))
DZ_model.add(Dense(alphabet_size, activation='softmax'))
optim = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
DZ_model.compile(loss=custom_loss, optimizer=optim)
early_stopping = EarlyStopping(monitor='loss', mode='min', min_delta=0.005, patience=3, verbose=1)
callbacks_list = [early_stopping]
DZ_model.fit(X, Y, epochs=num_epochs, batch_size=bs, verbose=1, shuffle=True, callbacks=callbacks_list)

## Compression

In [7]:
DZ_model = load_model('my_model.h5', compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
class ArithmeticEncoder(object):
    def __init__(self, bitlen):
        self.bit_prec = bitlen   #bit precision 
        self.max_range = 1 << self.bit_prec  #max range based on bit precision 2^bit_prec
        self.mask = self.max_range - 1  #max range index starting at 0
        self.renorm= self.max_range >> 1  #renormalization threshold
        self.second_mask = self.max_range >> 1
        self.low = 0  #initial low
        self.high = self.mask  #initial high
        self.s = 0  

    def update(self, sym, c):
        low = self.low   
        high = self.high
        range = high - low + 1
#         print(sym)
#         print(sym.shape )
        sym = int(sym)
        total = (c[-1])  #cumulative propabilities
        symlow = (c[sym])  
        symhigh = (c[sym+1] )
        

        newlow = low + symlow*range // total  #low in arithmetic integer
        newhigh = low + symhigh*range // total -1 #high in arithemtic integer
        self.low = int(newlow)
        self.high = int(newhigh)
        range = self.high - self.low 
#         print("sym:", sym)
#         print("symlow:", self.low)
#         print("symhigh:", self.high)
#         print("range:", range)
        
        #renormalization
        while((self.low ^ self.high) & self.renorm) == 0:
            self.low = (self.low << 1) 
            range = range << 1 | 1
            self.high = self.low + range
            self.s = self.s + 1          
#         print("renorm low:",self.low)
#         print("renorm range:",range)
#         print("renorm high:", self.high)
   
    def write(self, c, sym):
        self.update(c,sym)
        return [self.low, self.s]
        
    def finish(self):
        self.output.write(1)


# In[25]:


class ArithmeticDecoder(ArithmeticEncoder):
    def __init__(self,statesize,bitin,c):
        self.input = bitin
        self.bitstream = ([int(d) for d in str(self.input)])
        self.code = 0
        self.max_range = 1 << statesize
        self.renorm= self.max_range >> 1
        self.mask = self.max_range - 1
        self.stream = self.input[0:statesize]
        self.low = int(self.stream,2)
        self.t = statesize
        self.thresh = []
        self.total = (c[-1])
        self.cum = [c[i]/self.total for i in range(len(c))]
        self.thresh = [round((self.cum[i])*self.mask) for i in range(len(self.cum))]
        
    def decode(self, c):
#         print("low:",self.low
        c = [c[i]/self.total for i in range(len(c))]
#         print("cum::", c)
        for i in range(len(self.thresh)-1):
            if((self.low < self.thresh[i+1]) & (self.low >= self.thresh[i])):
                sym = i
#                 print("encoded value:",self.low)
#                 print("threshold:", self.thresh)
#                 print("decoded value:", sym)
                rangenew = int(self.thresh[i+1] - self.thresh[i])
#                 print("range:", rangenew)
#                 print("low:", self.thresh[i])
                while (rangenew < self.renorm):
                    rangenew = rangenew << 1 
                    self.low = self.low << 1
                    self.thresh[i] = int(self.thresh[i]) << 1
#                     print("renorm val;",self.low)
#                     print("renorm range;",rangenew)
                    self.thresh[i+1] = rangenew + self.thresh[i]
#                     print("renorm low:" ,self.thresh[i])
                self.thresh = [(c[j]*(self.thresh[i+1]-self.thresh[i]))+self.thresh[i] for j in range(len(self.cum))]
                break
        return sym

In [0]:
alphabet_size = 4
def cumul_d(prob):
    return np.cumsum(prob, axis = 1)  


In [155]:
bitprecision = 64
l = int(X.shape[0] / bs) * bs 
print(l)
print((X.shape[0])-time_steps)

#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
c = np.zeros(alphabet_size+1,  dtype = np.uint64)
c[1:] = np.cumsum(prob*10000000 + 1)        
bitstream = []
enc = ArithmeticEncoder(bitprecision)

num_iters = int((len(X)+time_steps)/bs)
ind = np.array(range(bs))*num_iters
prob = DZ_model.predict(X[ind,:], batch_size=bs)

#Encoding
for i in (range(bs)):
    for j in range(min(time_steps, num_iters)):
        new = enc.write(X[ind[i], j], c.tolist())
        
low_final = new[0] #final low value 
s_final = new[1]  #count of renormalizations
ind = ind + 1
bitstream_init = format(int(low_final), 'b') 
while len(bitstream_init) < (s_final+bitprecision):
    bitstream_init = '0' + bitstream_init
print(len(bitstream_init))

#convert low to bitstream, ensure proper number of bits 
cumul = np.zeros((bs, alphabet_size + 1), dtype = np.uint64)
cumul_enc = np.zeros((bs*(num_iters-time_steps), alphabet_size + 1), dtype = np.uint64)
for k in (range(num_iters - time_steps)):
    prob = DZ_model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob*10000000 + 1, axis = 1)
    cumul_enc[k*bs:k*bs+bs,:] = cumul
    for i in range(bs):
        enc.write(Y_raw[ind[i]], cumul[i,:].tolist())
    ind = ind + 1
    
print(cumul.shape)
print(cumul_enc.shape)
low_final = new[0] #final low value 
s_final = new[1]  #count of renormalizations
ind = ind + 1
#convert low to bitstream, ensure proper number of bits 
bitstream = format(int(low_final), 'b')  
while len(bitstream) < (s_final+bitprecision):
    bitstream= '0' + bitstream

bitstream = bitstream_init + bitstream
encval = int(bitstream)//(2**bitprecision)
print("encoded value:", encval)
print("encoded bit length:", len(bitstream))



# symdec = []
# # print("C:", c)
# for symbols in range(len(X)):
#     symdec.append(dec.decode(cumul))
    
    
# print(X.shape)
# print("input stream:", X[:,0])
# print("decoded num stream:", symdec)
# print(len(symdec))




11520
11456
16448
(128, 5)
(3328, 5)
encoded value: ...
encoded bit length: 32896


In [158]:

 #START DECODER
series = np.zeros((bs,num_iters), dtype = np.uint8)  
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
c = np.zeros(alphabet_size+1,  dtype = np.uint64)
c[1:] = np.cumsum(prob*10000000 + 1)        
num_iters = int((len(X)+time_steps)/bs)
ind = np.array(range(bs))*num_iters
dec = ArithmeticDecoder(bitprecision, bitstream, c)

# for i in range(bs):
#   for j in range(min(num_iters,time_steps)):
#     series[i,j] = dec[i].decode(c)
# print(series.shape)
symdec = []
for symbols in range(bs):
    symdec.append(dec.decode(c.tolist()))
print(X.shape)
# print("decoded num stream:", symdec)
print(len(symdec))

cumul = np.zeros((bs, alphabet_size + 1), dtype = np.uint64)        
for k in (range(num_iters - time_steps)):
  c = cumul_enc[k*bs:k*bs+bs,:]
#     prob = DZ_model.predict(series[:,j:j+time_steps], batch_size=bs)
#     cumul[:,1:] = np.cumsum(prob*10000000 + 1, axis = 1)
  for i in range(bs):
    symdec.append(dec.decode(c[i,:].tolist()))
    
print(X.shape)
print("decoded num stream:", symdec)
print(len(symdec))





(11520, 64)
128
(11520, 64)
decoded num stream: [1, 3, 3, 2, 0, 0, 1, 3, 3, 2, 0, 0, 1, 3, 3, 2, 0, 0, 1, 3, 3, 2, 0, 0, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 1, 2, 1, 2, 0, 0, 1, 1, 2, 0, 2, 1, 2, 0, 1, 1, 2, 2, 1, 1, 1, 0, 2, 1, 0, 1, 2, 0, 1, 1, 1, 0, 3, 1, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 3, 0, 1, 0, 2, 2, 2, 0, 2, 2, 0, 2, 1, 0, 2, 0, 2, 0, 0, 1, 0, 2, 1, 3, 0, 1, 0, 2, 0, 2, 3, 2, 3, 0, 2, 3, 0, 0, 1, 3, 2, 0, 0, 1, 0, 2, 1, 0, 2, 1, 1, 0, 0, 2, 1, 3, 0, 1, 1, 1, 1, 0, 1, 2, 0, 1, 2, 1, 3, 0, 2, 2, 0, 3, 2, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 3, 3, 2, 1, 1, 2, 0, 0, 2, 1, 3, 2, 3, 2, 2, 1, 0, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 1, 1, 2, 1, 3, 2, 2, 1, 1, 2, 1, 2, 0, 2, 0, 0, 1, 2, 2, 1, 2, 1, 

In [0]:
 series_2d = np.zeros((bs,num_iters), dtype = np.uint8)
 print(series_2d.shape)

(128, 90)


In [0]:
cumul[:,0:]

In [0]:
np.asscalar(cumul[-1])


In [0]:
X[i,:].reshape(1,-1).shape

In [0]:
if (l > (len(X.shape[0])-timesteps)):
    prob = model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob, axis = 1)
    for i in range(bs):
            enc[i].write(cumul[i,:], y_original[ind[i]])
    ind = ind + 1
else:

In [0]:
DZ_model = load_model('my_model.h5', compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
#make an array of cumulative probabilities
c = []
for i in range(len(prob)+1):
    c.append(sum(prob[0:i]))

cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)  
for j in (range(num_iters - timesteps)):
    prob = model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob, axis = 1)
    for i in range(bs):
        new = enc.write(Y_raw[ind[i]], cumul[i,:].tolist())
    ind = ind + 1
    low_final = new[0] #fina low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b')  
while len(bitstream) < (s_final+bitprecision):
    bitstream = '0' + bitstream
    print("encoded bit length:", len(bitstream))


    #START DECODER
    dec = ArithmeticDecoder(bitprecision, bitstream, c)

    symdec = []
    for symbols in range(len(sym)):
        symdec.append(dec.decode())
    print("decoded num stream:", symdec)

    #convert numbers back to letters
    dec_stream = []
    for x, sym in enumerate(symdec): 
        if sym == 0:
            dec_stream.append('a')
        elif sym ==1:
            dec_stream.append('c')
        elif sym == 2:
            dec_stream.append('g')
        elif sym == 3:
            dec_stream.append('t')

    dec_stream = dec_stream[:-1] 
    print("decoded symbol stream:", dec_stream)

NameError: name 'timesteps' is not defined

In [0]:
cumul

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0,

In [0]:

print(X)
print(X[ind,:].shape)

[[0 3 3 ... 3 3 1]
 [3 3 1 ... 3 1 2]
 [3 1 2 ... 1 2 2]
 ...
 [3 0 0 ... 1 1 1]
 [0 0 3 ... 1 1 3]
 [0 3 0 ... 1 3 1]]
(128, 64)


In [0]:
cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)  
print(cumul[0,:].tolist())


[0, 0, 0, 0, 0]


In [0]:
print(sum(prob[1,:]))

1.0000000596046448


In [0]:
import arithmetic_coder

symbol stream input: ['a' 'c' 'g' 't' 't' 'a' 'c' 't' 'g' 'a' 't' 'a' 'a' 't' 'a' 'c' 'c' 'g'
 't']
num stream: [0, 1, 2, 3, 3, 0, 1, 3, 2, 0, 3, 0, 0, 3, 0, 1, 1, 2, 3]
probability: [0.5 0.2 0.2 0.1]
cumulative prob: [0, 0.5, 0.7, 0.8999999999999999, 0.9999999999999999]
encoded bit length: 106
decoded num stream: [0, 1, 2, 3, 3, 0, 1, 3, 2, 0, 3, 0, 0, 3, 0, 1, 1, 2, 3, 1]
decoded symbol stream: ['a', 'c', 'g', 't', 't', 'a', 'c', 't', 'g', 'a', 't', 'a', 'a', 't', 'a', 'c', 'c', 'g', 't']


In [0]:
prob = np.ones(alphabet_size)/alphabet_size
cumul = np.zeros(alphabet_size+1, dtype = float)
cumul[1:] = np.cumsum(prob)        

In [0]:
cumul

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [0]:
prob = np.ones(alphabet_size)/alphabet_size
cumul = np.zeros(alphabet_size+1, dtype = np.uint64)
cumul[1:] = np.cumsum(prob*10000000 + 1)   

In [0]:
cumul

array([       0,  2500001,  5000002,  7500003, 10000004], dtype=uint64)